In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import multiprocessing as mp
from torch.autograd import variable
from torch.distributions import Categorical
import gym
import random

In [2]:
ALPHA = .001
EPISODE = 100
EPSILON = 1

In [3]:
class A3C(nn.Module):
    def __init(self):
        super(A3C, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc_pi = nn.Linear(32, 2)
        self.fc_v = nn.Linear(32, 1)
    
    def pi(self, x):
        x = self.fc1(x)
        x = F.softmax(self.fc_pi(x), dim=0)
        return x
    
    def v(self, x):
        x = self.fc1(x)
        x = self.fc_v(x)
        return x

In [4]:
# 독립 agent 학습 환경
def train_worker(net):
    EPISODE = 100
    env = gym.make('CartPole-v1')
    optimizer = optim.Adam(net.parameters(), ALPHA)
    ep = 1
    
    while(ep < EPISODE):
        done = False
        state = env.reset()

        while(not done):
            s_buf, a_buf, r_buf = [], [], []
            
            # 일정 step 만큼 경험 쌓기
            for t in range(5):
                
                # action 선택
                action_prob = net.pi(torch.from_numpy(state).float())
                m = Categorical(action_prob)
                action = m.sample().item()

                # step 진행
                state_next, reward, done, _ = env.step(a)

                # episode history 쌓기
                s_buf.append(state)
                a_buf.append(action)
                r_buf.append(reward)
                
                # state 갱신
                state = state_next

                if(done):
                    break
                    
            discounted = 0
            dis_lst = []
            
            # discounted reward to go 계산
            for r in r_buf:
                discounted = r + GAMMA * discounted
                dis_lst.append(discounted)
            dis_lst.reverse()
            
            s_buf = torch.tensor(s_buf, dtype=torch.float)
            a_buf = torch.tensor(a_buf)
            r_buf = torch.tensor(r_buf)
            
            # Advantage 계산
            td_target = r_buf + GAMMA * net.v(s_buf)
            advantage = dis_lst - net.v(s_buf)
            
            # policy 계산
            pi = net.pi(s_buf, softmax_dim=1)
            pi_a = pi.gather(1, a_buf)
            
            # 통합 loss
            loss = -torch.log(pi_a) * advantage.detach() + F.mse_loss(net.v(s_buf), td_target.detach())
            
            optimizer.zero_grad()
            loss.mead().backward()
            optimizer.step()
    env.close()
    print("Training process reached maximum episode.")


In [6]:
if __name__ == '__main__':
    # 글로벌 네트워크 생성
    mp.set_start_method('spawn')
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()
    
    Global_net = A3C()
    Global_net.share_memory()
    processes = []

    # 프로세스 생성
    p = Pool(processes=5)
    p.map_async(train_worker, Global_net)

    
    '''
    for i in range(5):
        p = mp.Process(target=train_worker, args=(Global_net))
        p.start()
        processes.append(p)

    for proc in processes:
        proc.join()'''

RuntimeError: context has already been set